In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/train.csv')

age_to_num = {'> 2 Years': 2.5, '1-2 Year' : 1.5,'< 1 Year' : 0.5 }
df_train.loc[:,'Vehicle_Age_Number'] = df_train.loc[:,'Vehicle_Age'].copy().replace(to_replace=age_to_num)

damage_to_bool = {'Yes': 1, 'No' :0}
df_train.loc[:,'Vehicle_Damage_Boolean'] = df_train.loc[:,'Vehicle_Damage'].copy().replace(to_replace=damage_to_bool)

gender_to_bool = {'Male': 1, 'Female' :0}
df_train.loc[:,'Gender_Boolean'] = df_train.loc[:,'Gender'].copy().replace(to_replace=gender_to_bool)

df_train = df_train[df_train['Annual_Premium'] < 40000]

df_train

In [ ]:
#### WORK IN PROGRESS

xx = df_train['Policy_Sales_Channel'].value_counts().reset_index()

channel_other = xx.iloc[11:,:]['index'].reset_index(drop=True)

index = df_train['Policy_Sales_Channel'].isin(channel_other)

df_train.loc[index,'Policy_Sales_Channel'] = 0

df_train = pd.get_dummies(df_train,prefix='chan', columns=['Policy_Sales_Channel'])

df_train = pd.get_dummies(df_train,prefix='region', columns=['Region_Code'])

df_train.loc[:,'Annual_Premium'] = df_train['Annual_Premium']/max(df_train['Annual_Premium'])
df_train.loc[:,'Vintage'] = df_train['Vintage']/max(df_train['Vintage'])
df_train.loc[:,'Age'] = df_train['Age']/max(df_train['Age'])

df_train

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.linear_model import LogisticRegression

X = df_train.drop(labels =['Response', 'id', 'Vehicle_Age', 'Vehicle_Damage', 'Gender'], axis=1)
y = df_train.Response

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [ ]:
model = GradientBoostingClassifier(n_estimators = 200)
#model = LogisticRegression(C = 100, max_iter = 1000)
model.fit(X_train, y_train)

In [ ]:
# Test data in X_test

y_predict = model.predict(X_test)

In [ ]:
y_test.sum()

In [ ]:
y_predict.sum()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predict))